In [50]:
import pandas as pd
import numpy as np
import re

In [51]:
# pd.set_option('display.max_colwidth', None)

In [52]:
pd.reset_option('display.max_colwidth')

In [53]:
master_data = {
    "otodom_2023_master" : "../original_data_2023/otodom/ALL_DATA/gdansk_otodom_master_data.json",
}

for key, value in master_data.items():
    master_data[key] = pd.read_json(value)  

otodom_df = master_data["otodom_2023_master"]

In [54]:
df = otodom_df

In [55]:
df.shape

(4535, 12)

In [56]:
df.head()

,link,address,title,price,surface,rent_extra,rooms,deposit,floor,building_type,avilable_since,outside_are
0,https://www.otodom.pl/pl/oferta/mieszkanie-z-w...,"ul. Żabi Kruk, Śródmieście, Gdańsk, pomorskie",Mieszkanie z widokiem na Motławę!,2 500 zł,"39,27 m²",500 zł/miesiąc,2.0,3 000 zł,parter/8,None,2023-08-14,None
1,https://www.otodom.pl/pl/oferta/dwa-pokoje-ant...,"Oliwa, Gdańsk, pomorskie","Dwa pokoje+antresola, niezależna kuchnia/jadalnia",3 300 zł,65 m²,650 zł/miesiąc,2.0,3 300 zł,2/2,dom wolnostojący,None,None
2,https://www.otodom.pl/pl/oferta/pacific-wynaje...,"ul. Obrońców Wybrzeża 15, Przymorze Wielkie, G...",Pacific | wynajem | balkon | parking,3 600 zł,44 m²,860 zł/miesiąc,2.0,3 600 zł,None,None,None,None
3,https://www.otodom.pl/pl/oferta/apartament-2-p...,"ul. Angielska Grobla, Śródmieście, Gdańsk, pom...",Apartament 2-pokojowy prestiżowe osiedle River...,4 100 zł,"53,2 m²",934 zł/miesiąc,2.0,5 500 zł,5/5,apartamentowiec,None,None
4,https://www.otodom.pl/pl/oferta/mieszkanie-z-b...,"ul. Piotrkowska, Ujeścisko-Łostowice, Gdańsk, ...",Mieszkanie z balkonem | Świetna komunikacja,3 200 zł,"55,5 m²",700 zł/miesiąc,3.0,6 400 zł,3/3,blok,None,balkon


In [57]:
# Sopotpomorskie               26
# Gdańskigdańskipomorskie      11
# Kolbudygdańskipomorskie       4
# Gdyniapomorskie               3
# Żukowokartuskipomorskie       2
# Grudziądzkujawsko             1

otodom_gda_districts_mapping = {
    "Śródmieście": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    
    
#     "StareMiasto": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
#     "GłówneMiasto": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
#     "DolneMiasto": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
#     "StarePrzedmieście": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
#     "DługieOgrody": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
#     "WyspaSpichrzów": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    
    "Wielkie": "Przymorze",
    "Małe": "Przymorze",
    "Dwór": "Przymorze",
    
    "Południe": "Chelm - Lostowice - Poludnie",
    "Łostowice": "Chelm - Lostowice - Poludnie",
    "Chełm": "Chelm - Lostowice - Poludnie",
#     "Zakoniczyn": "Chelm - Lostowice - Poludnie",
#     "Kowale": "Chelm - Lostowice - Poludnie",
    
#     "Wrzeszcz": "Wrzeszcz",
    "Dolny": "Wrzeszcz",
    "Górny": "Wrzeszcz",
    
    "Brzeźno": "Brzezno",
    
    "Jelitkowo": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Żabianka": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Wejhera": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Tysiąclecia": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",

    
    "Siedlce": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Aniołki": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Młyniska": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Mickiewicza": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    
    "Jasień": "Jasien - Matarnia - Kokoszki - Osowa",
    "Osowa": "Jasien - Matarnia - Kokoszki - Osowa",
    "Kokoszki": "Jasien - Matarnia - Kokoszki - Osowa",
#     "Kiełpinek": "Jasien - Matarnia - Kokoszki - Osowa",
    "Matarnia": "Jasien - Matarnia - Kokoszki - Osowa",
    
    "Rozstaje": "Zaspa",
    "Młyniec": "Zaspa",
    
    "Suchanino": "Piecki Migowo - Suchanino - Morena",
    "Brętowo": "Piecki Migowo - Suchanino - Morena",
#     "Piecki": "Piecki Migowo - Suchanino - Morena",
    "Migowo": "Piecki Migowo - Suchanino - Morena",
#     "Niedźwiednik": "Piecki Migowo - Suchanino - Morena",
    
    "Letnica": "Letnica",
    
    "Oliwa": "Oliwa - Strzyza",
    "Strzyża": "Oliwa - Strzyza",
    
#     "Orunia": "Orunia",
    "Lipce": "Orunia",
    "Zachodnie": "Orunia",
    "Olszynka": "Orunia",
    
    "Port": "Przerobka - Stogi - Nowy Port",
    "Przeróbka": "Przerobka - Stogi - Nowy Port",
    "Stogi": "Przerobka - Stogi - Nowy Port",
    "Sobieszewska": "Przerobka - Stogi - Nowy Port",
    
#     np.nan: "unknown"
}

otodom_rooms_mapping = {
    3: 3,
    2: 2,
    1: 1,
    4: 4,
    5: 4,
    6: 4,
    7: 4
}

otodom_private_seller_mapping = {
    1.0: True,
    np.nan: False
}

otodom_furnished_mapping = {
    "Tak": True,
}


otodom_level_mapping = {
    'parter': 0,
    '1': 1, '2': 2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10,
    '11': 11,
    '12': 11,
    '13': 11,
    '14': 11,
    '15': 11,
    '16': 11,
    '17': 11,
    'poddasze': 11,
}

otodom_building_type_mapping = {
    "apartamentowiec": "apartment_building",
    "blok": "unit",
    "Unknown": "unknown",
    "kamienica": "tenement",
    "dom wolnostojący": "house",
    "szeregowiec": "house",
    "plomba": "unit",
}

In [58]:
dirty_data = df.copy()

In [59]:
df.isna().sum()

link                 0
address             43
title               43
price               43
surface             31
rent_extra        1302
rooms               31
deposit           1570
floor              228
building_type      921
avilable_since    3355
outside_are       1439
dtype: int64

In [60]:
df.columns

Index(['link', 'address', 'title', 'price', 'surface', 'rent_extra', 'rooms',
       'deposit', 'floor', 'building_type', 'avilable_since', 'outside_are'],
      dtype='object')

In [61]:
df.head()

,link,address,title,price,surface,rent_extra,rooms,deposit,floor,building_type,avilable_since,outside_are
0,https://www.otodom.pl/pl/oferta/mieszkanie-z-w...,"ul. Żabi Kruk, Śródmieście, Gdańsk, pomorskie",Mieszkanie z widokiem na Motławę!,2 500 zł,"39,27 m²",500 zł/miesiąc,2.0,3 000 zł,parter/8,None,2023-08-14,None
1,https://www.otodom.pl/pl/oferta/dwa-pokoje-ant...,"Oliwa, Gdańsk, pomorskie","Dwa pokoje+antresola, niezależna kuchnia/jadalnia",3 300 zł,65 m²,650 zł/miesiąc,2.0,3 300 zł,2/2,dom wolnostojący,None,None
2,https://www.otodom.pl/pl/oferta/pacific-wynaje...,"ul. Obrońców Wybrzeża 15, Przymorze Wielkie, G...",Pacific | wynajem | balkon | parking,3 600 zł,44 m²,860 zł/miesiąc,2.0,3 600 zł,None,None,None,None
3,https://www.otodom.pl/pl/oferta/apartament-2-p...,"ul. Angielska Grobla, Śródmieście, Gdańsk, pom...",Apartament 2-pokojowy prestiżowe osiedle River...,4 100 zł,"53,2 m²",934 zł/miesiąc,2.0,5 500 zł,5/5,apartamentowiec,None,None
4,https://www.otodom.pl/pl/oferta/mieszkanie-z-b...,"ul. Piotrkowska, Ujeścisko-Łostowice, Gdańsk, ...",Mieszkanie z balkonem | Świetna komunikacja,3 200 zł,"55,5 m²",700 zł/miesiąc,3.0,6 400 zł,3/3,blok,None,balkon


In [62]:
df = dirty_data.copy()

import re

def convert_numeric_to_float(convertable_string):
    return float(convertable_string.replace("zł", "").replace(" ", "").replace("\ndonegocjacji", "").replace(",",".").replace("2000EUR", "8938").replace("10000EUR", "44694").replace("/miesiąc", ""))

def last_camel_case(input_string):
    matches = re.findall(r'[A-ZĄĆĘŁŃÓŚŹŻ][a-ząćęłńóśźż]*', input_string)
    if matches:
        return matches[-1]
    else:
        return None
    
def clear_otodom(df: pd.DataFrame):
    df.dropna(subset=['address', 'price', 'surface', 'rooms'], inplace=True)
    
    df['floor'] = df['floor'].fillna(1001)
    df["building_type"] = df["building_type"].fillna("Unknown")
    
    df.drop(
        labels=[
            'avilable_since',
            'outside_are',
            'link',
            'title'
        ],
        inplace= True,
        axis= 1
    )
    
    # rent
    df = df.rename(mapper={"price": "rent"}, axis=1)
    df['rent'] = df['rent'].apply(convert_numeric_to_float)

    # rent_extra
#     df['rent_extra'] = df['rent_extra'].apply(convert_numeric_to_float)

#     # location
#     df = df.rename(mapper={"address": "location"}, axis=1)
#     df['location'] = df['location'].str.lstrip('Gdańsk').str.rstrip('pomorskie').str.replace(' ', '').replace(',', '')
#     df['location'] = df['location'].map(otodom_gda_districts_mapping)

    return df

In [72]:
df = clear_otodom(df)

KeyError: ['price']

In [73]:
df["building_type"].value_counts()

building_type
apartamentowiec     1868
blok                1320
Unknown              886
kamienica            339
dom wolnostojący      57
szeregowiec           17
plomba                 2
Name: count, dtype: int64

In [65]:
df.head()

,address,rent,surface,rent_extra,rooms,deposit,floor,building_type
0,"ul. Żabi Kruk, Śródmieście, Gdańsk, pomorskie",2500.0,"39,27 m²",500 zł/miesiąc,2.0,3 000 zł,parter/8,Unknown
1,"Oliwa, Gdańsk, pomorskie",3300.0,65 m²,650 zł/miesiąc,2.0,3 300 zł,2/2,dom wolnostojący
2,"ul. Obrońców Wybrzeża 15, Przymorze Wielkie, G...",3600.0,44 m²,860 zł/miesiąc,2.0,3 600 zł,1001,Unknown
3,"ul. Angielska Grobla, Śródmieście, Gdańsk, pom...",4100.0,"53,2 m²",934 zł/miesiąc,2.0,5 500 zł,5/5,apartamentowiec
4,"ul. Piotrkowska, Ujeścisko-Łostowice, Gdańsk, ...",3200.0,"55,5 m²",700 zł/miesiąc,3.0,6 400 zł,3/3,blok


In [66]:
df["deposit"].value_counts()

deposit
3 000 zł       384
4 000 zł       377
3 500 zł       284
5 000 zł       225
6 000 zł       110
              ... 
3 490 zł         1
3 929 zł         1
3 330 zł         1
1 700 zł         1
2 440,44 zł      1
Name: count, Length: 149, dtype: int64

In [67]:
df.shape

(4489, 8)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4489 entries, 0 to 4534
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   address        4489 non-null   object 
 1   rent           4489 non-null   float64
 2   surface        4489 non-null   object 
 3   rent_extra     3221 non-null   object 
 4   rooms          4489 non-null   float64
 5   deposit        2953 non-null   object 
 6   floor          4489 non-null   object 
 7   building_type  4489 non-null   object 
dtypes: float64(2), object(6)
memory usage: 315.6+ KB


In [69]:
df.isna().sum()

address             0
rent                0
surface             0
rent_extra       1268
rooms               0
deposit          1536
floor               0
building_type       0
dtype: int64

In [70]:
df['address'].str.replace('ul.', '').str.replace('Gdańsk, pomorskie', '').str.replace(',', '').str.replace(' ', '').apply(last_camel_case).value_counts()

address
Śródmieście                1342
Wielkie                     382
Małe                        296
Brzeźno                     286
Górny                       278
Dolny                       236
Letnica                     147
Oliwa                       146
Jasień                      144
Jelitkowo                   136
Południe                    128
Migowo                      115
Łostowice                   111
Rozstaje                    102
Siedlce                      90
Lipce                        75
Chełm                        61
Młyniec                      57
Żabianka                     44
Osowa                        31
Kokoszki                     30
Suchanino                    27
Sopotpomorskie               26
Strzyża                      23
Aniołki                      23
Wejhera                      23
Młyniska                     21
Port                         15
Dwór                         14
Stogi                        13
Przeróbka                    12


In [71]:
df.head()

,address,rent,surface,rent_extra,rooms,deposit,floor,building_type
0,"ul. Żabi Kruk, Śródmieście, Gdańsk, pomorskie",2500.0,"39,27 m²",500 zł/miesiąc,2.0,3 000 zł,parter/8,Unknown
1,"Oliwa, Gdańsk, pomorskie",3300.0,65 m²,650 zł/miesiąc,2.0,3 300 zł,2/2,dom wolnostojący
2,"ul. Obrońców Wybrzeża 15, Przymorze Wielkie, G...",3600.0,44 m²,860 zł/miesiąc,2.0,3 600 zł,1001,Unknown
3,"ul. Angielska Grobla, Śródmieście, Gdańsk, pom...",4100.0,"53,2 m²",934 zł/miesiąc,2.0,5 500 zł,5/5,apartamentowiec
4,"ul. Piotrkowska, Ujeścisko-Łostowice, Gdańsk, ...",3200.0,"55,5 m²",700 zł/miesiąc,3.0,6 400 zł,3/3,blok
